# Calculo de principales indicadores usando python

A continuación se presentan los coódigos para el cálculo de los principales indicadores de la ENIGH 2020. Estan escritos para que el usuario los ejecute sin necesidad de cambio.

Para realizar este ejercicio solo seran necesatios los módulos de **pandas** y **numpy**

## Set-up

In [2]:
# Importamos librerias que vamos a utilizar
import pandas as pd
import matplotlib.pyplot as plt
import os

pd.options.display.float_format = '{:,.2f}'.format

In [3]:
# Creamos directorios
data = '/home/jovyan/work/02_indicadores_desigualdad/00_enigh/00_data/'
outputs = '/home/jovyan/work/02_indicadores_desigualdad/00_enigh/03_outputs/'

### Ejecutamos script con funciones necesarias

In [4]:
%run 'functions.py' 

## Lectura de tablas de datos

Para leer las tablas que necesitamos para el analisis, se creo una funcíon, **enigh**, que nos ayuda a traernos direcatmente de la pagina de la enigh la tabla del año que necesitemos, sin necesidad de guardar el arhivo en nuestro espacio de trabajo.

En este ejercicio, tomaremos la tabla *concentrado hogar* del año *2020* y lo guardamos en un dataframe

In [5]:
df = enigh('concentradohogar','2020')

___

## 2. Cálculo de indicadores de ingreso
### 2.1. Ingreso corriente total promedio trimestral por hogar en deciiles de hogares y su coeficiente de GINI

A continuación, se presenta el código que calcula el ingreso corriente total promedio por hogar en deciles de hogares y su coeficiente de GINI.
El coeficiente de Gini es una medida de concentración del ingreso: toma valores entre cero y uno. Cuando el valor se acerca a uno, indica que hay mayor concentración del ingreso; en cambio cuando el valor del Gini se acerca a cero la concentración del ingreso es menor.

<blockquote><div style="background-color:#DCDCDC; color:#000000; font-style: normal; font-family: Georgia;">
Folioviv (Folio de la vivienda):

Consta de 10 dígitos: dos dígitos con la clave de la entidad federativa, uno con el ámbito (urbano, código diferente a 6; rural, código 6), cuatro dígitos del número consecutivo de la upm (unidad primaria de muestreo), un dígito de la decena de levantamiento y dos dígitos con un número consecutivo para la vivienda seleccionada.
</div></blockquote>


<blockquote><div style="background-color:#DCDCDC; color:#000000; font-style: normal; font-family: Georgia;">
Foliohog (Folio del hogar):

Dígito para identificar a los hogares en la vivienda: se asigna 1 al hogar principal (el que inicialmente
atendió la entrevista) o cuando en la vivienda solo habita un hogar; cuando se detecten varios hogares
(hogares adicionales) se les asignan dígitos del 2 al 5.
</div></blockquote>

In [ ]:
print('Total de Hogares: ', df.factor.sum())

print('Tamaño de los deciles: ',  df.factor.sum()/10 )

print('Promedio de integrantes por familia: {:.3} '.format(df.tot_integ.mean()))

#### Analisis de carga fiscal en los hogares
Queremos analizar la carga fiscal de los hogares, haremos una aproximación del cuanto pagan los hogares de IVA por decil, excluyendo alimentos que tienen tasa 0:
* foliovivi: folio de vivienda
* foliohog: folio hogar 
* ing_cor: Suma de los ingresos por trabajo, los provenientes de rentas, de transferencias, de estimación del alquiler y de otros ingresos.
* gasto_mon: Es la suma de los gastos regulares que directamente hacen los hogares en bienes y servicios para su consumo.
    * suma de alimentos, vesti_calz, vivienda, limpieza, salud, transporte, educa_espa y transf_gas
* alimentos: gasto en alimentos dentro y fuera del hogar
    * Alimentos excentos de IVA: carnes, pescado, leche, huevo, tuberculo, verduras y frutas
* vesti_calz: gasto en vestido y calzado
* vivienda: Gasto en vivienda, servicios de conservación, energía eléctrica y combustibles
    * La suma de alquiler, pred_cons, agua y energia
* limpieza: Gasto en artículos y servicios para la limpieza, cuidados de la casa, enseres domésticos y muebles, cristalería, utensilios domésticos y blancos.
    * La suma de cuidados, utensilios y enseres
* salud: Gasto en cuidados de la salud.
    * Gasto en cuidados de la salud.
    * Conceptos excentos de pagar IVA: medicina 
* transporte:
    * Concepto excento de pagar IVA: transporte público
* educa_espa: Gasto en servicios de educación, artículos educativos, artículos de esparcimiento y otros gastos esparcimiento.
    * La suma de educacion, esparci y paq_turist.

In [6]:
# Seleccionamos variables de interes para análisis
concentrado = df[['ing_cor', 'gasto_mon', 'alimentos', 'vesti_calz', 'vivienda', 'limpieza', 'salud', 'transporte', 
                  'educa_espa', 'personales','transf_gas', 'carnes', 'pescado', 'leche','huevo', 'tuberculo', 'verduras',
                  'frutas', 'medicinas', 'publico','factor']]

In [7]:
concentrado['decil']=weighted_qcut(concentrado['ing_cor'], concentrado['factor'], 10, labels=False)

/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


Una vez que creamos el dataframe con las variables que vamos a utilizar, tenemos que restar a gasto monerario corriente (gasto_mon) las variables de los conceptos excentos de iva.

Y para finalizar, vamos a multiplicar las variables por el factor de expansión para tener valores nacionales.

In [8]:
concentrado['gasto_final'] = concentrado['gasto_mon']

# Lista para quitar del gasto monetario corriente total
exc= ['carnes', 'pescado', 'leche','huevo', 'tuberculo', 'verduras', 'frutas', 'medicinas', 'publico', 'transf_gas']

for var in exc:
    concentrado['gasto_final'] = concentrado['gasto_final'] - concentrado[var]

In [9]:
# lista para quitar del concepto alimentos
exc1= ['carnes', 'pescado', 'leche','huevo', 'tuberculo', 'verduras', 'frutas']

for var in exc1:
    concentrado['alimentos'] = concentrado.loc[:,'alimentos'] - concentrado.loc[:,var]

# Quitamos conceptos a salud
concentrado['salud'] = concentrado['salud'] - concentrado['medicinas']

# Quitamos conceptos a transporte
concentrado['transporte'] = concentrado['transporte'] - concentrado['publico']

In [10]:
# Se crea una variable de entidad federativa y de total de integrantes
ls=['ing_cor', 'gasto_mon', 'gasto_final','alimentos', 'vesti_calz', 'vivienda', 'limpieza', 'salud', 'transporte', 
    'educa_espa', 'personales', 'transf_gas']

for var in ls:
    concentrado[var]= concentrado[var]*concentrado['factor']

concentrado['integrantes']= df['tot_integ']*concentrado['factor']
# concentrado['entidad']=df.loc[:,('folioviv')].apply(str).str[1:3]

Para crear los deciles de ingreso, utilizamos la función *weighted_qcut* definida al inicio del notebook para obtener los deciles ponderados por el factor de expansión

In [11]:
deciles_final=concentrado[['decil','ing_cor','gasto_mon','gasto_final', 'alimentos', 'salud', 'transporte', 'vesti_calz', 
             'limpieza', 'vivienda','educa_espa','personales']].groupby(by='decil').sum()

In [31]:
deciles_final.to_excel(outputs + 'data/deciles_desagregado.xlsx')

In [12]:
deciles_final['total_decil']=deciles_final[['alimentos', 'salud', 'transporte', 'vesti_calz', 'limpieza', 'vivienda', 
                                            'educa_espa','personales']].sum(axis=1)

In [13]:
a = (deciles_final['alimentos']/deciles_final['total_decil'])*100
b = (deciles_final['salud']/deciles_final['total_decil'])*100

test=pd.concat([a,b], axis=1)

In [1]:
test.plot(kind="bar",stacked=True,figsize=(10,8))
plt.legend(loc="lower left",bbox_to_anchor=(0.8,1.0))
plt.show()

NameError: name 'test' is not defined